In [7]:
import json
import os
from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys
import optparse
import shutil
from mri_preproc.paths import hemond_data
from utils import dice_score
from pathlib import Path
import re
import nibabel as nib
import numpy as np

In [8]:
curr_dir = os.getcwd()
dataroot = Path("/mnt/h/3Tpioneer_bids")
work_dir = Path("/home/srs-9/Projects/ms_mri/training_work_dirs/cp_work_dir_choroid1")  # the algorithm working directory generated by AlgoGen/BundleGen
ensemble_out_dir = work_dir / "ensemble_output" / "3Tpioneer_bids"
save_dir = Path("/mnt/h/3Tpioneer_bids_predictions")

In [9]:
for sub_dir in ensemble_out_dir.glob("sub*"):
    subid = re.match(r"(sub-ms\d{4})", sub_dir.name)[1]
    ses_dirs = [Path(item.path) for item in os.scandir(sub_dir) if "ses" in item.name]

    for ses_dir in ses_dirs:
        sesid = re.match(r"(ses-\d+)", ses_dir.name)[1]
        orig_label = dataroot / subid / sesid / "choroid_t1_flair.nii.gz"
        pred_label = ses_dir / "t1_flair_ensemble.nii.gz"

        seg1 = nib.load(orig_label).get_fdata()
        seg2 = nib.load(pred_label).get_fdata()

        # Calculate Dice score
        score = dice_score(seg1, seg2)
        print(f'Dice Score: {score}')

Dice Score: 0.6565690946221919
Dice Score: 0.8159203980099502
Dice Score: 0.7923165325890659
Dice Score: 0.76162030987493
Dice Score: 0.7822267620020429
Dice Score: 0.836217234880932
Dice Score: 0.797744687003036


In [10]:
dataset = hemond_data.get_t1_flair_3Tpioneer_bids(dataroot)

In [11]:
# infile = dataset.find_scan(subid="ms1001", ses="20180323").image
infile = dataset[0].image
infile.relative_to(dataroot)

PosixPath('sub-ms1001/ses-20170215/t1_flair.nii.gz')

In [12]:
images = []
for i in range(78):
    infile: Path = dataset[i].image
    images.append({"image": str(infile.relative_to(dataroot))})

In [13]:
# datalist = {
#     "testing": [{"image": infile.name}]
# }

datalist = {
    "testing": images
}

datalist_file = "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f)


task = {
  "name": "infer_choroid",
  "task": "segmentation",
  "modality": "MRI",
  "datalist": "datalist.json",
  "dataroot": str(dataroot),
}

task_file = os.path.join(curr_dir, "task.json")
with open(task_file, "w") as f:
    json.dump(task, f)


In [14]:
input_cfg = "task.json"  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)


In [15]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage", 
    "output_dir": save_dir, 
    "data_root_dir": dataroot, 
    "output_postfix": "choroid_pred", 
    "separate_folder": False}
                               
pred = ensemble(pred_param = {"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/78 [00:00<?, ?it/s]monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

2024-09-03 22:19:08,414 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1001/ses-20170215/t1_flair_choroid_pred.nii.gz


Image save path not returned.
Ensembling (rank 0)...:   1%|▏         | 1/78 [02:59<3:49:58, 179.20s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is dep

2024-09-03 22:22:08,100 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1001/ses-20180323/t1_flair_choroid_pred.nii.gz


Ensembling (rank 0)...:   3%|▎         | 2/78 [05:59<3:47:33, 179.65s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp

...

Ensembling (rank 0)...:  97%|█████████▋| 76/78 [3:37:42<05:47, 173.70s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.am

2024-09-04 01:56:45,867 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1038/ses-20180328/t1_flair_choroid_pred.nii.gz


Ensembling (rank 0)...:  99%|█████████▊| 77/78 [3:40:37<02:53, 173.87s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.am

2024-09-04 01:59:06,073 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1038/ses-20181002/t1_flair_choroid_pred.nii.gz


Ensembling (rank 0)...: 100%|██████████| 78/78 [3:42:57<00:00, 171.50s/it]
